<a href="https://colab.research.google.com/github/Wyw021214/ISP_autism/blob/main/data_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install -q nilearn nibabel pandas numpy matplotlib requests

In [28]:
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nilearn import image

##Abide

In [15]:
summary_url = "https://fcp-indi.s3.amazonaws.com/data/Projects/ABIDE_Initiative/Phenotypic_V1_0b_preprocessed1.csv"
phenotypic_file = "phenotypic_data.csv"

!wget -O {phenotypic_file} {summary_url}

phenotypic_data = pd.read_csv(phenotypic_file)
phenotypic_data.head()

--2024-12-05 16:25:50--  https://fcp-indi.s3.amazonaws.com/data/Projects/ABIDE_Initiative/Phenotypic_V1_0b_preprocessed1.csv
Resolving fcp-indi.s3.amazonaws.com (fcp-indi.s3.amazonaws.com)... 52.217.227.233, 16.15.217.139, 3.5.21.148, ...
Connecting to fcp-indi.s3.amazonaws.com (fcp-indi.s3.amazonaws.com)|52.217.227.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 449443 (439K) [application/octet-stream]
Saving to: ‘phenotypic_data.csv’

phenotypic_data.csv 100%[===================>] 438.91K   579KB/s    in 0.8s    

2024-12-05 16:25:51 (579 KB/s) - ‘phenotypic_data.csv’ saved [449443/449443]



,Unnamed: 0.1,Unnamed: 0,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,...,qc_notes_rater_1,qc_anat_rater_2,qc_anat_notes_rater_2,qc_func_rater_2,qc_func_notes_rater_2,qc_anat_rater_3,qc_anat_notes_rater_3,qc_func_rater_3,qc_func_notes_rater_3,SUB_IN_SMP
0,0,1,50002,1,50002,PITT,no_filename,1,1,16.77,...,NaN,OK,NaN,fail,ic-parietal-cerebellum,OK,NaN,fail,ERROR #24,1
1,1,2,50003,2,50003,PITT,Pitt_0050003,1,1,24.45,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
2,2,3,50004,3,50004,PITT,Pitt_0050004,1,1,19.09,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
3,3,4,50005,4,50005,PITT,Pitt_0050005,1,1,13.73,...,NaN,OK,NaN,maybe,ic-parietal-cerebellum,OK,NaN,OK,NaN,0
4,4,5,50006,5,50006,PITT,Pitt_0050006,1,1,13.37,...,NaN,OK,NaN,maybe,ic-parietal slight,OK,NaN,OK,NaN,1


In [16]:
base_url = "https://fcp-indi.s3.amazonaws.com/data/Projects/ABIDE_Initiative/Outputs"
"""
[pipeline] = ccs | cpac | dparsf | niak
[strategy] = filt_global | filt_noglobal | nofilt_global | nofilt_noglobal
[file identifier] = the FILE_ID value from the summary spreadsheet
[derivative] = alff | degree_binarize | degree_weighted | dual_regression | ...
               eigenvector_binarize | eigenvector_weighted | falff | func_mask | ...
               func_mean | func_preproc | lfcd | reho | rois_aal | rois_cc200 | ...
               rois_cc400 | rois_dosenbach160 | rois_ez | rois_ho | rois_tt | vmhc
[ext] = 1D | nii.gz"""

pipeline = "cpac"
strategy = "filt_global"
derivative = "rois_aal"
file_ext = "1D"

file_ids = phenotypic_data["FILE_ID"].dropna().head(5)
download_links = [
    f"{base_url}/{pipeline}/{strategy}/{derivative}/{file_id}_{derivative}.{file_ext}"
    for file_id in file_ids
]

download_path = "/content/abide_aal"
os.makedirs(download_path, exist_ok=True)

for link in download_links:
    file_name = link.split("/")[-1]
    file_path = os.path.join(download_path, file_name)


    if not os.path.exists(file_path):
        print(f"Downloading: {file_name}")
        os.system(f"wget -q -P {download_path} {link}")
    else:
        print(f"Skipping: {file_name} (already exists)")

Downloading: no_filename_rois_aal.1D
Downloading: Pitt_0050003_rois_aal.1D
Downloading: Pitt_0050004_rois_aal.1D
Downloading: Pitt_0050005_rois_aal.1D
Downloading: Pitt_0050006_rois_aal.1D


In [19]:
print("\nVerifying downloaded AAL files...")
aal_files = [f for f in os.listdir(download_path) if f.endswith(".1D")]
print(f"Downloaded {len(aal_files)} AAL parcellation files:")
for f in aal_files:
    print(f)


Verifying downloaded AAL files...
Downloaded 4 AAL parcellation files:
Pitt_0050003_rois_aal.1D
Pitt_0050004_rois_aal.1D
Pitt_0050006_rois_aal.1D
Pitt_0050005_rois_aal.1D


In [23]:
if aal_files:
    print("\nLoading and visualizing the first AAL file...")
    first_aal_file = os.path.join(download_path, aal_files[0])
    aal_data = np.loadtxt(first_aal_file)

    # Check the data dimensions
    print(f"Shape of AAL data (timepoints x ROIs): {aal_data.shape}")


Loading and visualizing the first AAL file...
Shape of AAL data (timepoints x ROIs): (196, 116)


##HCP

In [36]:
from google.colab import files

# Upload the Aspera private key
uploaded = files.upload()

In [33]:
!file /content/hcp_data/hcp_sample.nii.gz

/content/hcp_data/hcp_sample.nii.gz: HTML document, ASCII text, with very long lines (3909)


In [32]:
hcp_img = image.load_img(hcp_file)
print(f"HCP data shape: {hcp_img.shape}")

ImageFileError: File /content/hcp_data/hcp_sample.nii.gz is not a gzip file

In [39]:
from nilearn import datasets

sample_data = datasets.fetch_development_fmri(n_subjects=1)
func_file = sample_data['func'][0]  # Path to the NIfTI file

print(f"Sample fMRI file: {func_file}")

[get_dataset_dir] Dataset created in /root/nilearn_data/development_fmri

[_add_readme_to_default_data_locations] Added README.md to /root/nilearn_data/development_fmri

[get_dataset_dir] Dataset created in /root/nilearn_data/development_fmri/development_fmri

[fetch_single_file] Downloading data from https://osf.io/yr3av/download ...

[fetch_single_file]  ...done. (3 seconds, 0 min)

...

[fetch_single_file] Downloading data from https://osf.io/download/5c8ff3df4712b400183b7092/ ...

[fetch_single_file]  ...done. (4 seconds, 0 min)

[fetch_single_file] Downloading data from https://osf.io/download/5c8ff3e04712b400193b5bdf/ ...

[fetch_single_file]  ...done. (4 seconds, 0 min)

Sample fMRI file: /root/nilearn_data/development_fmri/development_fmri/sub-pixar123_task-pixar_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz


In [40]:
from nilearn import datasets, image
from nilearn.input_data import NiftiLabelsMasker

aal_atlas = datasets.fetch_atlas_aal()

masker = NiftiLabelsMasker(labels_img=aal_atlas.maps, standardize=True)

hcp_img = image.load_img(func_file)
hcp_time_series = masker.fit_transform(hcp_img)
print(f"Extracted time series shape: {hcp_time_series.shape}")


[get_dataset_dir] Dataset found in /root/nilearn_data/aal_SPM12

Extracted time series shape: (168, 116)
